In [1854]:
import datetime

from yahoo_fin import stock_info as si
import numpy as np
import pandas as pd
import time
import os

cache_dir = 'cache_dir'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [1855]:
def get_new_ticker(ticker_, interval):
    new_ticker = pd.DataFrame
    file_ticker = os.path.join(cache_dir,ticker_ + '_' + time.strftime("%Y-%m-%d") + '_' + interval + ".csv")
    if not os.path.isfile(file_ticker):
        new_ticker = si.get_data(ticker_, interval=interval)
        if "date" not in new_ticker.columns:
            new_ticker["date"] = new_ticker.index
        new_ticker.to_csv(file_ticker, index=False)

    if os.path.isfile(file_ticker):
        new_ticker = pd.read_csv(file_ticker,index_col='date')
        new_ticker.rename(columns={'close':ticker_}, inplace=True)
        new_ticker.drop(['open', 'high','low','volume','ticker','adjclose'], axis='columns', inplace=True)

    return new_ticker

def portfolio(tickers_list, interval):
    df_ticker = pd.DataFrame
    for ticker in tickers_list:
        if not df_ticker.empty:
            df_ticker = pd.concat([df_ticker, get_new_ticker(ticker, interval) ], axis=1)
        else:
            df_ticker = pd.concat([get_new_ticker(ticker, interval) ], axis=1)

    return df_ticker

# VOO	QQQ	KWEB	VNQ	TLT	BAR	COMT	VIXY	BTC-USD
# 

In [1856]:
tickers = [
    'BTC-USD',
    'VOO', #Vanguard S&P 500 ETF
    'QQQ',
    'KWEB',
    'VNQ',
    'TLT', #iShares 20+ Year Treasury Bond ETF (TLT)
    'BAR',
    # # 'GLD',
    'COMT', # iShares GSCI Commodity Dynamic Roll Strategy ETF (COMT)
    # # 'DBC', # Invesco DB Commodity Index Tracking Fund (DBC)
    # 'VIXY',
    '^VIX',

    ]

# 14%	14%	5%	22%	13%	10%	10%	5%	7%
# VOO	QQQ	KWEB	VNQ	TLT	BAR	COMT	VIXY	BTC-USD

df = portfolio(tickers,'1d') #“1d”, “1wk”, “1mo”
df.tail(5)
df.to_csv("df.csv")

In [1857]:
df = df.loc['2021-11-01':]
df.head()

,BTC-USD,VOO,QQQ,KWEB,VNQ,TLT,BAR,COMT,^VIX
date,,,,,,,,,
2021-11-01,61004.406250,423.049988,387.429993,50.299999,109.580002,146.440002,17.790001,37.630001,16.410000
2021-11-02,63226.402344,424.690002,389.049988,48.400002,110.099998,147.089996,17.740000,37.700001,16.030001
2021-11-03,62970.046875,427.250000,393.179993,49.049999,110.470001,145.570007,17.600000,36.959999,15.100000
2021-11-04,61452.230469,429.250000,398.220001,47.840000,109.419998,147.100006,17.809999,36.740002,15.440000
2021-11-05,61125.675781,430.709991,398.600006,46.840000,109.980003,149.300003,18.040001,37.220001,16.480000


In [1858]:
df.replace('', np.nan, inplace=True)
df = df.dropna()

In [1859]:
# percent = {'QQQ': 25,'TLT':25,'GLD':25,'^VIX':25}
percent = {0:60,
           1:0,
           2:0,
           3:0,
           4:0,
           5:0,
           6:0,
           7:0,
           8:40
           }
start_summ , begin = 1000000 , 1000000
sum_per = 0
for j in range(len(percent)):
    sum_per += percent[j]
print(sum_per)

100


In [1860]:
rows = len(df.index)
columns = len(df.columns)

In [1861]:
# print("rows: " + str(rows) + " columns: " + str(columns))
df['summ'] = 1000000
for i in range(rows):
    row_summ = 0
    if i > 0:
        start_summ = df.iloc[i-1:i,columns:columns+1].values
    for j in range(columns):
        if i > 0 and j < columns:
            # start_summ = df.iloc[i-1:i,columns:columns+1].values
            # print(df.iloc[i-1:i,j:j+1].values)
            # print("summ:" + str(start_summ) + "---")
            # start_summ = df.iloc[i-1:i,columns:columns+1].values
            pre_count = start_summ * percent[j]/100/df.iloc[i-1:i,j:j+1].values
            pre_summ = pre_count * df.iloc[i:i+1,j:j+1].values
            row_summ = row_summ + pre_summ
            df.iloc[i:i+1,columns:columns+1] = row_summ

# df.head(10)
print((row_summ/begin-1)*100)

[[-3.8263262]]


In [1862]:
df.to_excel("portfolio.xlsx")
df

,BTC-USD,VOO,QQQ,KWEB,VNQ,TLT,BAR,COMT,^VIX,summ
date,,,,,,,,,,
2021-11-01,61004.406250,423.049988,387.429993,50.299999,109.580002,146.440002,17.790001,37.630001,16.410000,1.000000e+06
2021-11-02,63226.402344,424.690002,389.049988,48.400002,110.099998,147.089996,17.740000,37.700001,16.030001,1.012591e+06
2021-11-03,62970.046875,427.250000,393.179993,49.049999,110.470001,145.570007,17.600000,36.959999,15.100000,9.866294e+05
2021-11-04,61452.230469,429.250000,398.220001,47.840000,109.419998,147.100006,17.809999,36.740002,15.440000,9.812467e+05
2021-11-05,61125.675781,430.709991,398.600006,46.840000,109.980003,149.300003,18.040001,37.220001,16.480000,1.004556e+06
2021-11-08,67566.828125,431.230011,398.059998,48.040001,109.889999,149.020004,18.110001,37.480000,17.219999,1.086112e+06
2021-11-09,66971.828125,429.720001,395.320007,47.880001,110.080002,150.960007,18.180000,37.770000,17.780001,1.094502e+06
2021-11-10,64995.230469,426.260010,389.510010,48.980000,109.570000,148.199997,18.379999,37.310001,18.730000,1.098512e+06
2021-11-11,64949.960938,426.399994,390.589996,51.529999,109.750000,147.979996,18.480000,37.459999,17.660000,1.072951e+06
